In [ ]:
# %pip install llama-index-vector-stores-faiss

In [ ]:
# MODEL_REPO = "thenlper/gte-small"
TEXT_GENERATION_MODEL_REPO = "mistralai/Mixtral-8x7B-Instruct-v0.1"
# TEXT_GENERATION_MODEL_REPO = "HuggingFaceH4/zephyr-7b-alpha"
# MODEL_REPO = "Writer/camel-5b-hf"
from my_settings import LLMSettings

model = LLMSettings(
    model_name=TEXT_GENERATION_MODEL_REPO,
    context_window=10000,
    max_new_tokens=10000,
    max_length=10000,
    temperature=0.85,
    query_wrapper_prompt="""
    Below is an instruction that describes a task. 
    You'r helpful AI assisant given the task to help people seeking advise.
    Use the following pieces of context to answer the question at the end.
    The text provided to you is about indian constitution and legal processes and provide information in a lawful manner
    You should prefer information which are more related to asked question.
    Make sure to rely on information from text only and not on questions to provide accurate responses.
    When you find particular answer in the context useful, make sure to cite it in the your answer.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Each answer you generate should contain a section numbers you found useful.
    You can only use the given to you to answer the question.
    Generate concise answers and relevant data related to the asked question.
    You must represent the answer in proper format such as make points highlight some major information.
    don't attach your created quetions. if you don't get answer from the given text just say i don't know and terminate answering.
    if you get answer from the text than write all about the asked quetion and relevant data related to it.
    don't use your own knowledge just use the provided text to answer the question.  
    also don't provide such example quetions with answer. just give the answer asked by the user in proper manner.
    ### Instruction:\n{query_str}\n\n### Response:
""",
)

model._set_huggingface_embedding(local=True)
model._setup_huggingface_llm_inference_api()
model._configure_mongodb()

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
)

In [ ]:
docs = SimpleDirectoryReader(input_files=["./data/IPC_186045.pdf"]).load_data()

In [ ]:
import faiss

# dimensions of "BAAI/bge-small-en"
d = 384
faiss_index = faiss.IndexFlatL2(d)
faiss_index

In [ ]:
from llama_index.vector_stores.faiss import FaissVectorStore
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    docs, storage_context=storage_context
)

In [ ]:
# save index to disk
index.storage_context.persist()

In [ ]:
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
)
# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage"
)
index = load_index_from_storage(storage_context=storage_context)

In [ ]:
# retriever = index.as_retriever(similarity_top_k=3)
# nodes = retriever.retrieve("flower")

# for node in nodes:
#     print(node)

In [ ]:
query_engine = index.as_query_engine(
    kwargs={"early_stopping": True, "min_length": 2000, "max_tokens": 5000}
)

In [ ]:
from IPython.display import Markdown, display

response = query_engine.query("tell me about rioting !")
display(Markdown(f"{response}"))